How to create a Deployment
==========================

In this notebook, we show you how to create a Deployment with 3 ReplicaSets. These ReplicaSets are owned by the Deployment and are managed by the Deployment controller. We would also learn how to carry out RollingUpdate and RollBack to new and older versions of the deployment.

In [1]:
!pip install kubernetes

In [2]:
import os

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/opt/app/config/authcreds.json'

In [4]:
from kubernetes import client, config

### Load config from default location

In [5]:
#config.load_kube_config()
config.load_kube_config(config_file='/opt/app/config/kube.config')
extension = client.ExtensionsV1beta1Api()

### Create Deployment object

In [6]:
deployment = client.ExtensionsV1beta1Deployment()

### Fill required Deployment fields (apiVersion, kind, and metadata)

In [7]:
deployment.api_version = "extensions/v1beta1"
deployment.kind = "Deployment"
deployment.metadata = client.V1ObjectMeta(name="nginx-deployment")

### Add Pod template in .spec.template section

In [8]:
#template = client.V1PodTemplateSpec()
#template.metadata = client.V1ObjectMeta(labels={"app": "nginx"})
# Configureate Pod template container
container = client.V1Container(
        name="nginx",
        image="nginx:1.7.9",
        ports=[client.V1ContainerPort(container_port=80)])
# Create and configurate a spec section
template = client.V1PodTemplateSpec(
        metadata=client.V1ObjectMeta(labels={"app": "nginx"}),
        spec=client.V1PodSpec(containers=[container]))
# Create the specification of deployment
spec = client.ExtensionsV1beta1DeploymentSpec(
        replicas=3,
        template=template)

### A Deployment also needs a .spec section

In [9]:
#spec = client.ExtensionsV1beta1DeploymentSpec(
#        replicas=3,
#        template=template)

### Pod template container description

In [10]:
#container = client.V1Container(
#        name="nginx",
#        image="nginx:1.7.9",
#        ports=[client.V1ContainerPort(container_port=80)])

In [11]:
spec.template.spec.containers = [container]
deployment.spec = spec

### Create Deployment

In [12]:
extension.create_namespaced_deployment(namespace="default", body=deployment)

{'api_version': 'extensions/v1beta1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 5, 22, 15, 9, 58, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'initializers': None,
              'labels': {'app': 'nginx'},
              'name': 'nginx-deployment',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '847042',
              'self_link': '/apis/extensions/v1beta1/namespaces/default/deployments/nginx-deployment',
              'uid': '31427c0c-5dd2-11e8-9a5c-42010a8000ab'},
 'spec': {'min_ready_seconds': None,
          'paused': None,
          'progress_deadline_seconds': None,
          'replicas': 3,
          'revision_history_limit'

In [20]:
v1 = client.CoreV1Api()
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print("%s\t%s\t%s" %
          (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

10.8.0.6	default	nginx-1-97dd4c589-phv5q
10.8.0.8	default	nginx-deployment-6bd4859cdb-2hqh7
10.8.2.10	default	nginx-deployment-6bd4859cdb-4qwgm
10.8.2.9	default	nginx-deployment-6bd4859cdb-przk7
10.8.1.4	kube-system	event-exporter-v0.1.7-5c4d9556cf-hb4cx
10.8.2.2	kube-system	fluentd-gcp-v2.0.9-6h9bm
10.8.0.2	kube-system	fluentd-gcp-v2.0.9-bpvd6
10.8.1.2	kube-system	fluentd-gcp-v2.0.9-lk8gw
10.8.1.6	kube-system	heapster-v1.4.3-86c6587454-mhq92
10.8.2.3	kube-system	kube-dns-778977457c-jngvq
10.8.0.5	kube-system	kube-dns-778977457c-kftwp
10.8.0.4	kube-system	kube-dns-autoscaler-7db47cb9b7-fmjkv
10.240.0.47	kube-system	kube-proxy-gke-cluster-1-default-pool-45398d44-6ftc
10.240.0.48	kube-system	kube-proxy-gke-cluster-1-default-pool-45398d44-8nph
10.240.0.46	kube-system	kube-proxy-gke-cluster-1-default-pool-45398d44-pj3k
10.8.1.5	kube-system	kubernetes-dashboard-768854d6dc-rwzzv
10.8.1.3	kube-system	l7-default-backend-6497bcdb4d-249t2
